# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for k, v in obj.items():
    print(f'{k:10} = "{v}"')

home_page  = "https://github.com/pypa/sampleproject"
keywords   = "sample setuptools development"
license    = "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
import re
import pandas as pd
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
for i in obj:
    p=re.compile(r"['1-9']{1}$")
    print(p.findall(i))

['1']
['4']
['3']


3. Разбейте текст формулировки задачи 2 на слова.

In [3]:
k='Написать регулярное выражение,которое позволит найти номера групп студентов'
p=re.compile(r"[\w]+")
p.findall(k)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [4]:
import csv
recipes=pd.read_csv('recipes_sample.csv')
d=recipes[['id', 'minutes']].sample(5)
c=len(str(max(recipes['id'])))
print('|    id     |  minutes  |')
print('|-----------------------|')
for i, r in d.iterrows():
    print(f"|{r['id']:^{c+4}} | {r['minutes']:^{c+4}}|")

|    id     |  minutes  |
|-----------------------|
|  311313   |     75    |
|  434814   |     70    |
|  18985    |     45    |
|  329542   |     75    |
|  208412   |     90    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [5]:
import requests
from bs4 import BeautifulSoup

with open('steps_sample.xml') as f:
    steps = BeautifulSoup(f, 'xml')

In [6]:
recipes=pd.read_csv('recipes_sample.csv')

In [7]:
def show_info(inp: int):
    for i in steps.find_all('recipe'):
        if int(i.find('id').get_text()) == inp:
            d = [] 
            for j in i.find_all('step'):
                step = j.get_text()
                d.append(step)
                
    string = recipes[recipes['id']==inp]
    title = string.iloc[0]['name']
    res = f'"{title}"'
    k = 0
    for i in d:
        res = res+f'{k}. {i}'+'\n'
        k+=1
    res = res+'--'*20+'\n' 
    
    minutes = string.iloc[0]['minutes'] 
    author = string.iloc[0]['contributor_id']
    
    res = res+f'Автор: {author}'+'\n'+f'Среднее время приготовления: {minutes}'
    return print(res)

In [8]:
show_info(170895)

"leeks and parsnips  sauteed or creamed"0. clean the leeks and discard the dark green portions
1. cut the leeks lengthwise then into one-inch pieces
2. melt the butter in a medium skillet , med
3. heat
4. add the garlic and fry 'til fragrant
5. add leeks and fry until the leeks are tender , about 6-minutes
6. meanwhile , peel and chunk the parsnips into one-inch pieces
7. place in a steaming basket and steam 'til they are as tender as you prefer
8. i like them fork-tender
9. drain parsnips and add to the skillet with the leeks
10. add salt and pepper
11. gently sautee together for 5-minutes
12. at this point you can serve it , or continue on and cream it:
13. in a jar with a screw top , add the half-n-half and arrowroot
14. shake 'til blended
15. turn heat to low under the leeks and parsnips
16. pour in the arrowroot mixture , stirring gently as you pour
17. if too thick , gradually add the water
18. let simmer for a couple of minutes
19. taste to adjust seasoning , probably an additio

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
t=re.compile(r'\d+\s(?:hour|hours|minute|minutes)')
res=[]
for i in steps.find_all('recipe'):
    if int(i.find('id').get_text())==25082:
        for j in i.find_all('step'):
            res=t.findall(j.get_text())
            if res!=[]:
                print(res)

['20 minute']
['10 minute']
['2 hour']
['10 minute']
['20 minute', '30 minute']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [10]:
p=re.compile(r'^this[\w\s]*,\s?but\b')
k=0
res=[]
for i,r in recipes.iterrows():
    res=p.findall(str(r['description']))
    if res != []:
        k+=1
        if k<=3:
            print(k, str(r['description']))
print(f'Количество подходящих рецептов: {k}')

1 this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
2 this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
3 this is kind of similar to some of the other versions out there, but it is the best and easiest i have found
Количество подходящих рецептов: 132


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [19]:
k=1
for i in steps.find_all('recipe'):
    if int(i.find('id').get_text())==72367:
        for j in i.find_all('step'):
            res=re.sub(r'(\d+)\s/\s(\d+)', r'\1/\2', j.get_text())
            print(f'{k}. {res}')
            k+=1

1. mix butter , flour , 1/3 c
2. sugar and 1-1/4 t
3. vanilla
4. press into greased 9" springform pan
5. mix cream cheese , 1/4 c
6. sugar , eggs and 1/2 t
7. vanilla beating until fluffy
8. pour over dough
9. combine apples , 1/3 c
10. sugar and cinnamon
11. arrange on top of cream cheese mixture and sprinkle with almonds
12. bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [63]:
from nltk.tokenize import word_tokenize
words=[]
for i in steps.find_all('recipe'):
    for j in i.find_all('step'):
        words.extend([word.lower() for word in word_tokenize(j.get_text()) if word.isalpha()])
len(set(words))

14926

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [64]:
from nltk.tokenize import sent_tokenize
recipes['sentences']=recipes['description'].apply(lambda x: len(sent_tokenize(str(x))))
h=recipes.sort_values('sentences', ascending=False).head(5)
h

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,sentences
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,24
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,23
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,23


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
